<a href="https://colab.research.google.com/github/gitanujjain/pyspark_practice/blob/main/Spark_explode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar -xf spark-3.5.3-bin-hadoop3.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
## Suppose you have a table Called log_data with the following column : log_id ,user_id ,action and timestamp.write a sql query to calculate the number of actions performed by each user in the last 7 days
data = [
  (1, 101, 'login', '2023-09-05 08:30:00'),
  (2, 102, 'click', '2023-09-06 12:45:00'),
  (3, 101, 'click', '2023-09-07 14:15:00'),
  (4, 103, 'login', '2023-09-08 09:00:00'),
  (5, 102, 'logout', '2023-09-09 17:30:00'),
  (6, 101, 'click', '2023-09-10 11:20:00'),
  (7, 103, 'click', '2023-09-11 10:15:00'),
  (8, 102, 'click', '2023-09-12 13:10:00')
]

df= spark.createDataFrame(data)

In [ ]:
from pyspark.sql.functions import col,current_date as current__date

df.filter((df.filter(col("timestamp")< current__date() and col("timestamp")> current__date()-7).groupby(user_id).count("user_id").collect())) & col("_4")> current__date()-7).groupby("_2").count("_2").collect()

PySparkValueError: [CANNOT_CONVERT_COLUMN_INTO_BOOL] Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [ ]:
df=spark.read.format("csv").option("header","true").load("/content/sample_data/sample_csv")
from pyspark.sql.functions import explode, split
df=df.withColumn("genres",split("genres", pattern='\|'))
df=df.withColumn("year",split("title", pattern='\(')[1])
df=df.withColumn("year",split("year", pattern='\)')[0])
df.collect()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/sample_data/sample_csv.

In [ ]:
df=df.withColumn("genres", explode("genres"))
print(df.count())
df=df.dropDuplicates()
print(df.count())

11
10


In [ ]:
print(spark)

## **Differnce between reducebykey and reduce**
**reduceByKey** is an transformation as it is used when working with key-value pairs. Imagine you have data in the form of (key, value) tuples, and you want to group it by keys and then perform some aggregation on the associated values.

**reduce** is a more general-purpose transformation. It can be applied to any RDD. reduce is used when you want to perform a reduction operation across all elements.

### Key Differences:
**Applicability:**
*   reduce: Applies to any RDD, not necessarily a pair RDD.
*   reduceByKey: Applies specifically to pair RDDs, where each element is a key-value pair.

**Operation Scope:**
*  reduce: Aggregates all elements of the RDD into a single result.
*  reduceByKey: Aggregates values for each key separately, producing a pair RDD with unique keys and reduced values.

**Usage:**
*  reduce: Used for global aggregation, irrespective of keys.
*  reduceByKey: Used for aggregating values based on keys, often in the context of grouping or summarizing data.

In [ ]:
a=[("car",2),("car",3),("car",4),("car1",2),("car1",3),("car1",4)]

rdd=spark.sparkContext.parallelize(a)
rdd.reduceByKey(lambda x,y:x+y).collect()

[('car', 9), ('car1', 9)]

In [ ]:
a=[1,2,3,4,5,6,7,8,9,10]
rdd1=spark.sparkContext.parallelize(a)
rdd1.reduce(lambda x+y:x+y).collect()

### Differnece between map and flatmap
**map Transformation:**
*  The map transformation applies a given function to each element of an RDD or DataFrame and returns a new RDD or DataFrame where each element corresponds to the result of applying the function to the corresponding element of the original RDD or DataFrame.
*  The output of the map transformation has a one-to-one mapping between input and output elements.
*  In other words, if you start with an RDD or DataFrame of n elements, the map transformation will produce a new RDD or DataFrame with the same number of elements.
*  The function applied by map can produce a single output element for each input element.

**2. flatMap Transformation:**

The flatMap transformation applies a function to each element of an RDD or DataFrame and returns a new RDD or DataFrame where each input element can be mapped to zero or more output elements. In other words, the output elements are flattened.
The function applied by flatMap can return a sequence or iterable of values for each input element, and these values will be concatenated into the output RDD or DataFrame.
The flatMap transformation is useful when you want to perform operations that generate multiple output elements for each input element, such as tokenizing text or exploding arrays.
**In summary:**
Use map when you want a one-to-one transformation, where each input element corresponds to exactly one output element.
Use flatMap when you want to perform a transformation that can generate zero or more output elements for each input element, and you want to flatten the results into a single RDD or DataFrame.
The choice between map and flatMap depends on your specific data processing requirements and whether you need to flatten the results or not.

In [ ]:
a=[1,2,3,4,5,6]
rdd2=spark.sparkContext.parallelize(a)
rdd3=rdd2.map(lambda x:x+2)
rdd3.collect()

[3, 4, 5, 6, 7, 8]

In [ ]:
a=[1,2,3,4,5,6]
rdd2=spark.sparkContext.parallelize(a)
rdd4=rdd2.flatMap(lambda x:x+2)
rdd4.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 73.0 failed 1 times, most recent failure: Lost task 0.0 in stage 73.0 (TID 67) (633e605a01af executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/content/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/content/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/content/spark-3.5.3-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
TypeError: 'int' object is not iterable

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.Pytho